Begin by importing necessary libraries.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

Define numerical methods to solve ODEs

In [2]:
def EuMaR(func, lamb, X0, lamb0, r, params=None, tstart=0, tend=1, steps=10, sigma = 0.03):
    """
    Euler-Maruyama Method for solving SDEs for R-tipping toy model.
    
    func := function, ex.dx/dt = ax + b == params[0] * x + params[1],
    lamb := function, parameter for R-tipping toy model,
    X0 := float, initial position,
    lamb0 := float, initial lambda value,
    r := numpy array, array of r values being inspected,
    params := array, list of parameters for func,
    tstart / tend := float, starting time / ending time,
    steps := int, no. of iterations,
    sigma := float, scale for amount of noise in system.
    """
    
    ## Initialise time vector
    ##  Calculate dt for Euler Method
    t = np.linspace(tstart,tend,steps)
    dt = abs(t[0]-t[1])
    
    ## Init. solution vector.
    ##  set initial value
    sol = np.ones([steps])
    sol[0] = X0
    lamb_i = lamb0
    
    ## Begin Euler-Maruyama method
    for i in range(1, steps):
        params[0] = lamb_i
        temp = sol[i-1] + func(sol[i-1], t=t[i-2], params=params) * dt +  sigma * np.random.random() * (np.random.randint(0,1)*2-1)
        
        
        sol[i] = temp
        lamb_i = lamb_i + lamb(r, t=t[i-2]) * dt
        
        ## Attempt to remedy float overflow
        if temp > 0:
            return sol, i
        
    ## Return Euler-Maruyama solution to SDE
    return sol, i

Defining 2-d ODE which can have an R-tipping point

In [3]:
f = lambda x,t,params: (x + params[0])**2 - params[1]
lamb = lambda r,t: r

Interactable widget for looking at the shape of an R-tipping system.

In [4]:
@interact(mu=(0,2,0.01), rstart=(0,2,0.01), rend=(0,2,0.01), X0=(-2,2,0.1), lamb0=(-5,5,0.1), tend=(0,1000,1), dt=(0.01,2,0.01), sigma=(0,2,0.001), n=(10,200,10))
def g(mu, X0, lamb0, rstart, rend,tend,dt,sigma,n):

    r = np.linspace(rstart,rend,n)
    
    steps = int(tend/dt)
    
    params = np.array([lamb0, mu])
    t = np.linspace(0,tend,steps)
    tot = 0
    for i in range(n):
        a, end = EuMaR(f, lamb, X0, lamb0, r=r[i], params=params, tend=tend, steps=steps, sigma=sigma)

        
        plt.plot(t[:end], a[:end])

    #plt.ylim(-3,2)
    plt.xlim(1.5,3.6)
    plt.show()

interactive(children=(FloatSlider(value=1.0, description='mu', max=2.0, step=0.01), FloatSlider(value=0.0, des…

# Generating R-tipping solutions

Follow mostly the same format as B-tipping

In [5]:
P = 100 ## No. of samples

## Initial parameters
X0 = 0
lamb0 = 0
mu = 1.5
params = np.array([lamb0, mu])

## Define r range
r = np.linspace(1.51,1.55,P)

## Amount of noise in system
sigma = 0.005


tend = 100
dt = 0.1
steps = int(tend/dt)

## Initialise data array
data = np.zeros((P, steps+1))

## Generate systems
for i in range(P):
    data[i,1:], end = EuMaR(f, lamb, X0, lamb0, r=r[i], params=params, tend=tend, steps=steps, sigma=sigma)
    if end < steps-1:
        data[i,0] = 1

sum(data[:,0])

62.0

Saves solutions for `Deep Learning.ipynb`

In [6]:
np.savetxt("data_rtip.csv", data, delimiter=',')

Interactable widget for user to look at solutions

In [7]:
@interact(k=(0,len(data)-1,1))
def g(k):
    plt.plot(data[k,1:])
    plt.show()

interactive(children=(IntSlider(value=49, description='k', max=99), Output()), _dom_classes=('widget-interact'…